# Toxic Text Part 2: spaCy


Detecting Insults in Social Commentary

Data from Wikipedia 



# Setup

## Python Library Imports


Resources:
- [pool]()

In [1]:
import pandas as pd
import numpy as np

from collections import Counter
import re
import random

# scikit learn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# tqdm & time
from tqdm.auto import tqdm
import time

%load_ext autoreload
%autoreload 2

## spaCy Setup & Imports

In [2]:
# spaCy Imports
import spacy

# from spacy.lang.en import English
# spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

# nlp = spacy.load('en_core_web_lg')

In [4]:
from spacy.util import minibatch, compounding
from spacy import displacy
from spacy.tokens import Doc
# from spacy.training import Example # downgraded to 2 from 3

## Load data from 2-2 Pickle File

In [9]:
%%time
# load from pickle file 2-3
'''
last load time:

CPU times: user 2min 59s, sys: 1min 9s, total: 4min 9s
Wall time: 4min 47s
'''
path2_2 = "../data/toxic_2-2.pkl"

toxic_df = pd.read_pickle(path2_2)

AttributeError: Can't get attribute 'unpickle_morphology' on <module 'spacy.morphology' from '/opt/anaconda3/lib/python3.7/site-packages/spacy/morphology.cpython-37m-darwin.so'>

In [10]:
toxic_df.info()

NameError: name 'toxic_df' is not defined

In [ ]:
# Convert training text and training outcomes into a list of tuples

toxic_df["tuples"] = toxic_df.apply(lambda row: (row['comment_text'], row['toxic']), axis=1)

In [ ]:
toxic_df['tuples'][0]

# Simple Train Test Split

As our process should first determine whether the text is toxic or not toxic, we'll make a simplified stratified train test split, ensuring our balance of toxic and non toxic rows are proportionally distributed.

For now, we won't be too concerned with the proportion of sub-categories, as our first step will be to filter not toxic from toxic, then run parallel operations for each toxic sub-category, as toxic sub-categories are not mutually exclusive.

## Stratified Split maintaining ratio of toxic to not toxic texts


In [ ]:
# check current columns
toxic_df.columns

In [ ]:
# split df into X(independent) and y(depenendent) groups
ind_cols = ['comment_text', 'doc_per_row', 'lemma_raw', 'uppercase_proportion', 'tuples']

X = toxic_df[ind_cols]
y = toxic_df.drop(columns=ind_cols)

print(f"X columns: {X.columns}\ny columns:{y.columns}")

In [ ]:
# Train Test Split. Stratified on y['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=42, 
                                                    stratify=y['toxic'])

# Stratified K Fold

- [SKF docs](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#)  

In [ ]:
tiny_df = toxic_df.sample(20)

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3,
                      random_state=42,
                      shuffle=True)
print(skf)

skf.get_n_splits(toxic_df['comment_text'], toxic_df['toxic'])

# for tr_in, test_in in skf.split(toxic['comment_text'], toxic['toxic']):
#     print(tr_in, test_in)


train_indx, test_indx = next(skf.split(toxic_df['comment_text'], toxic_df['toxic']))

# spaCy

Let's try out spaCy, a nlp processing library!

- https://course.spacy.io/en/chapter1
- [text classification with spaCy](https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/) 
- [customized list of stopwords](https://spacy.io/usage/linguistic-features#stop-words)  
- [Split Series into list of sentences](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.cat.html)  
- [contractions](https://theslaps.medium.com/cant-stand-don-t-want-contractions-with-spacy-39715cac2ebb)  


In [ ]:
# # spaCy Imports
# import spacy

# # from spacy.lang.en import English
# # spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
from spacy.training import Example

In [ ]:
# ! pip install last version of spaCy
! pip install 'spacy<3'

In [ ]:
# # check version

! python -m spacy info

## Establish spaCy Pipeline

"spaCy's components are supervised models for text annotations, meaning hey can only learn to reproduce examples, not guess new labels from raw text."

Code is modified from tutorial here:

Resource:
https://www.machinelearningplus.com/nlp/custom-text-classification-spacy/

In [ ]:
# nlp = spacy.load('en_core_web_lg')

In [ ]:
# # add binary textcat model
# # # textcat: binary: toxic or not toxit
# textcat = nlp.add_pipe("textcat", last=True)

# # # # We'll add the second later
# # # # textcat_multilabel: what type of toxic if toxic?
# # nlp.add_pipe("textcat_multilabel", last=True)

# nlp.pipe_names

In [ ]:
# # adding labels to textcat
# textcat.add_label("NOT TOXIC")
# textcat.add_label("TOXIC")

In [ ]:
# tiny_df['tuples']

# def t_nt_training_format(s):
#     '''
#     toxic, not toxic training format
    
#     takes series of tuples, 
#         str: text, int (0 or 1)
#     retunrs
#     '''
#     s_lst = s.tolist()
    
#     texts, labels = zip(*s)
    
#     cats = [{"TOXIC": bool(y), "NOT TOXIC": not bool(y)} for y in labels]
    
#     text = [i for i in texts]
    
# #     print(list(zip(text, cats)))
    
#     return list(zip(text, [{'cats': i} for i in cats]))
    
    
# #     for i in range(len(cats)):
# #         print(cats[i])
    
    
# t_nt_training_format(tiny_df['tuples'])[0]

In [ ]:
print(f"X columns: {X.columns}\ny columns:{y.columns}")

In [ ]:
# # Disabling other components
# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
# with nlp.disable_pipes(*other_pipes):  # only train textcat
#     optimizer = nlp.begin_training()

In [ ]:
spacy.util.minibatch?

# Try new, blank spaCy model

In [ ]:
nlp_2 = spacy.blank('en')

# textcat_2 = nlp_2.add_pipe('textcat')

In [ ]:
textcat_2.add_label("TOXIC")
textcat_2.add_label("NOT TOXIC")

In [ ]:
toxic_df['doc_per_row'][0].vocab

In [ ]:
# from spacy.tokens import Doc
# from spacy.training import Example


def txt_and_cat(txt_series, cat_series):
        
    # convert each series or series slice to list
    t = txt_series.tolist()
    c = cat_series.tolist()
    
    # format categories
    c = [{"TOXIC": bool(y), "NOT TOXIC": not bool(y)} for y in c]
    c = [{'cats': i} for i in c]
    
    docs = list(zip(t, c))
    
    return docs

[Article for v3](https://medium.com/analytics-vidhya/building-a-text-classifier-with-spacy-3-0-dd16e9979a)

In [ ]:
# ! ls ../models/base_config.cfg
# ! python -m spacy init fill-config ../models/base_config.cfg config.cfg --diff

In [ ]:
# ! python -m spacy validate

In [ ]:
txt = toxic_df['doc_per_row'][train_indx]
cat = toxic_df['toxic'][train_indx]

train_docs = txt_and_cat(txt, cat)

test_txt = toxic_df['doc_per_row'][test_indx]
test_cat = toxic_df['toxic'][test_indx]

test_docs = txt_and_cat(test_txt, test_cat)

In [ ]:
# this should be the correct format expected by the trainer
print(len(train_docs), len(test_docs))

print(train_docs[0][1])

print([i[1] for i in train_docs[:5]])

In [ ]:
test_txt_lst = [i[0] for i in test_docs]
test_cat_lst = [i[1] for i in test_docs]

In [ ]:
def sort(sub_li): 
  
    # reverse = True (So resulting_list = list(first_list)rts in Descending  order) 
    # key is set to sort using second element of  
    # sublist lambda has been used 
    return(sorted(sub_li, key = lambda x: x[1],reverse=True))  

# run the predictions on each sentence in the evaluation  dataset, and return the metrics
def evaluate(tokenizer, textcat, test_texts, test_cats ):
    docs = (tokenizer(text) for text in test_texts)
    preds = []
    for i, doc in enumerate(textcat.pipe(docs)):
        #print(doc.cats.items())
        scores = Sort(doc.cats.items())
        #print(scores)
        catList=[]
        for score in scores:
            catList.append(score[0])
        preds.append(catList[0])
        
    labels = ['TOXIC', 'NOT TOXIC']
    
    print(classification_report(test_cats,preds,labels=labels))

In [ ]:
print(textcat_2.labels)

In [ ]:
print(nlp_2.pipe_names)
other_pipes = nlp_2.pipe_names[1:6]
print(other_pipes)

In [ ]:
def train_spacy(train_data, iterations,test_texts,test_cats, model_arch, dropout = 0.3, model=None,init_tok2vec=None):
    ''' Train a spacy NER model, which can be queried against with test data
   
    train_data : training data in the format of (sentence, {cats: ['positive'|'negative'|'neutral']})
    labels : a list of unique annotations
    iterations : number of training iterations
    dropout : dropout proportion for training
    display_freq : number of epochs between logging losses to console
    '''
    
#     nlp = spacy.load("en_core_web_lg")
    

#     # add the text classifier to the pipeline if it doesn't exist
#     # nlp.create_pipe works for built-ins that are registered with spaCy
#     if "textcat" not in nlp.pipe_names:
#         textcat = nlp.create_pipe(
#             "textcat", config={"exclusive_classes": True, "architecture": model_arch}
#         )
#         nlp.add_pipe(textcat, last=True)
        
#     # otherwise, get it, so we can add labels to it
#     else:
#         textcat = nlp.get_pipe("textcat")

#     # add label to text classifier
#     textcat.add_label("positive")
#     textcat.add_label("negative")
#     textcat.add_label("neutral")

    # get names of other pipes to disable them during training
#     pipe_exceptions = ["textcat", "trf_wordpiecer", "trf_tok2vec"]
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp_2.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp_2.begin_training()
        if init_tok2vec is not None:
            with init_tok2vec.open("rb") as file_:
                textcat_2.model.tok2vec.from_bytes(file_.read())
        print("Training the model...")
        print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        batch_sizes = compounding(16.0, 64.0, 1.5)
        for i in range(iterations):
            print('Iteration: '+str(i))
            start_time = time.clock()
            losses = {}
            # batch up the examples using spaCy's minibatch
            random.shuffle(train_data)
            batches = minibatch(train_data, size=batch_sizes)
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=dropout, losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the test data 
                evaluate(nlp.tokenizer, textcat, test_texts,test_cats)
            print ('Elapsed time'+str(time.clock() - start_time)+  "seconds")
        with nlp.use_params(optimizer.averages):
            modelName = model_arch+"TweetClassification"
            filepath = "C:\\TweetSenitment\\"+modelName+"\\"
            nlp.to_disk(filepath)
    
    return nlp
    

In [ ]:
# Train (and save) the Text categorization model with BOW
nlp_2 = train_spacy(train_docs, 10, test_txt_lst, test_cat_lst, "bow")